In [74]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import pprint as pp

In [75]:
base_url = 'https://hh.ru'
search = '/search/vacancy'

In [76]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}

In [77]:
def zp(text):
    text = text.replace('\u202f', '')
    text_list = text.split()
    ot = '0'
    do = '0'
    if text.find('от') != -1:
        ot = text_list[text_list.index('от')+1]
    if text.find('до') != -1:
        do = text_list[text_list.index('до')+1]
    if text.find('–') != -1: 
        ot = text_list[text_list.index('–')-1]
        do = text_list[text_list.index('–')+1]
    val = text_list[-1]
    
    return ot, do, val

#### Введите вакансию:

In [78]:
inp_vacancy = 'Системный администратор'

In [79]:
max_page = 28

In [80]:
res_list = []
for num_page in np.arange(max_page):
    params = {'area':'1', 'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'page': num_page, 'hhtmFrom': 'vacancy_search_list'}
    response = requests.get(base_url + search, headers = headers, params = params)
    dom = BeautifulSoup(response.text, 'html.parser')
    conteiners = dom.find_all('div', {'class':"vacancy-serp-item"})
    for vac in conteiners:
        link_tag = vac.find('a', {'data-qa':"vacancy-serp__vacancy-title"}, recursive = 'False') 
        money = vac.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if money is None:
            res_list = np.append(res_list, [link_tag.text, link_tag['href'], '---', '0', '0', '---'])
        else:
            ot, do, val = zp(money.text)
            res_list = np.append(res_list, [link_tag.text, link_tag['href'], money.text, ot, do, val])    
            
result_df = pd.DataFrame(res_list.reshape((int(len(res_list)/6),6)), columns = ['Вакансия', 'Ссылка', 'Зарплата', 'От', 'До', 'Валюта'])  
result_df[['От', 'До']] = result_df[['От', 'До']].astype(int)

In [81]:
result_df.head(10)

,Вакансия,Ссылка,Зарплата,От,До,Валюта
0,IT-специалист/Системный администратор,https://hh.ru/vacancy/52593175?from=vacancy_se...,90 000 – 90 000 руб.,90000,90000,руб.
1,Начинающий системный администратор macOS / Linux,https://hh.ru/vacancy/52605239?from=vacancy_se...,от 85 000 руб.,85000,0,руб.
2,Системный администратор web-серверов (Linux),https://hh.ru/vacancy/52469788?from=vacancy_se...,от 100 000 руб.,100000,0,руб.
3,Системный администратор,https://hh.ru/vacancy/51883839?from=vacancy_se...,70 000 – 70 000 руб.,70000,70000,руб.
4,Инженер сопровождения / Системный администратор,https://hh.ru/vacancy/49318535?from=vacancy_se...,---,0,0,---
5,Ведущий системный администратор,https://hh.ru/vacancy/51315269?from=vacancy_se...,---,0,0,---
6,Системный администратор в поиск Яндекс,https://hh.ru/vacancy/52349868?from=vacancy_se...,до 50 000 руб.,0,50000,руб.
7,Системный администратор сетей связи,https://hh.ru/vacancy/50539871?from=vacancy_se...,---,0,0,---
8,Системный администратор,https://hh.ru/vacancy/51416560?from=vacancy_se...,от 80 000 руб.,80000,0,руб.
9,Системный администратор,https://hh.ru/vacancy/52568168?from=vacancy_se...,150 000 – 300 000 руб.,150000,300000,руб.
